In [46]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [63]:
training_epochs = 15
batch_size = 100

In [64]:
tf.reset_default_graph() 

Y = tf.placeholder(tf.float32, [None,10])
X = tf.placeholder(tf.float32, [None,784])

keep_prob = tf.placeholder(tf.float32)

X_img = tf.reshape(X, [-1, 28, 28, 1])

# L1 Img In Shape = (?,28,28,1)
W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
L1 = tf.nn.conv2d(X_img,W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') #stride가 2이기 때문에 14x14가 나온다
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# L2 Img In Shape = (?,14,14,1)
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
L2 = tf.nn.conv2d(L1,W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# L3 Img In Shape = (?,7,7,1)
W3 = tf.Variable(tf.random_normal([3,3,64,128],stddev=0.01))
L3 = tf.nn.conv2d(L2,W3, strides=[1,1,1,1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3 = tf.reshape(L3, [-1,128*4*4])

# 
W4 = tf.get_variable("W4",shape=[128*4*4,625],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3,W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5",shape=[625,10],initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4,W5)+b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob:0.7}
        c, _, = sess.run([cost,optimizer], feed_dict=feed_dict)
        avg_cost += c/total_batch
    print('Epoch : ', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch :  0001 cost = 0.511738709
Epoch :  0002 cost = 0.097943136
Epoch :  0003 cost = 0.069977737
Epoch :  0004 cost = 0.058757773
Epoch :  0005 cost = 0.050669702
Epoch :  0006 cost = 0.046996860
Epoch :  0007 cost = 0.040294747
Epoch :  0008 cost = 0.036272645
Epoch :  0009 cost = 0.034705411
Epoch :  0010 cost = 0.031970280
Epoch :  0011 cost = 0.030183099
Epoch :  0012 cost = 0.028042696
Epoch :  0013 cost = 0.027622270
Epoch :  0014 cost = 0.024984584
Epoch :  0015 cost = 0.025272341


In [76]:
prediction = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1))
test1 =tf.cast(prediction, tf.float32)
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
print(accuracy)

ac,t = sess.run([accuracy,test1],feed_dict={X:mnist.test.images,Y:mnist.test.labels,keep_prob:1})
print('Accuracy : ',ac)

Tensor("Mean_6:0", shape=(), dtype=float32)
Accuracy :  0.9933


In [75]:
import numpy
numpy.set_printoptions(threshold=numpy.nan)
print(t.shape)

(10000,)
